## Experiments notebook
This is a Notebook for experiments before adding methods to `scrape_data.py`.

In [25]:
import requests
import json

date = '22/08/2021'
response = requests.post(
    'https://wb2server.congreso.gob.pe/regvisitastransparencia/filtrar',
    data=json.dumps({
            'fechaDesde': '2021-09-02',
            'fechaHasta': '2021-09-02',
        })
)
response.encoding = 'utf-8-sig'
response.json()
# response.text

[{'empleado': 'ARAGON CARREÑO, LUIS ANGEL',
  'cargo': 'CONGRESISTA',
  'motivo': 'NORMAL',
  'idVisita': 6586706,
  'fila': 1,
  'fechaVisitaRecepcion': '02/09/21 08:48',
  'entidadVisitanteNombreCompleto': 'Sardon Loayza, Edmundo Chale',
  'entidadVisitanteTipoDocumento': 'DNI/LE',
  'entidadVisitanteDocumento': '29632116',
  'entidad': None,
  'centroCostoNombre': 'ARAGON CARREÑO LUIS ANGEL',
  'ubicacion': 'LUIS A. SANCHEZ - FERNANDO BELAUNDE TERRY    :    303 - CONG. ARAGON CARREÑO LUIS ANGEL',
  'fechaVisitaTermino': None,
  'edificio': 'LUIS A. SANCHEZ - FERNANDO BELAUNDE TERRY'},
 {'empleado': 'VERGARA MENDOZA, ELVIS HERNAN',
  'cargo': 'CONGRESISTA',
  'motivo': 'NORMAL',
  'idVisita': 6586707,
  'fila': 2,
  'fechaVisitaRecepcion': '02/09/21 08:50',
  'entidadVisitanteNombreCompleto': 'MENDOZA GARCIA, BLANCA ELIZABETH',
  'entidadVisitanteTipoDocumento': 'DNI/LE',
  'entidadVisitanteDocumento': '07239456',
  'entidad': None,
  'centroCostoNombre': 'VERGARA MENDOZA ELVIS HERNA

In [26]:
response.json()[0]

{'empleado': 'ARAGON CARREÑO, LUIS ANGEL',
 'cargo': 'CONGRESISTA',
 'motivo': 'NORMAL',
 'idVisita': 6586706,
 'fila': 1,
 'fechaVisitaRecepcion': '02/09/21 08:48',
 'entidadVisitanteNombreCompleto': 'Sardon Loayza, Edmundo Chale',
 'entidadVisitanteTipoDocumento': 'DNI/LE',
 'entidadVisitanteDocumento': '29632116',
 'entidad': None,
 'centroCostoNombre': 'ARAGON CARREÑO LUIS ANGEL',
 'ubicacion': 'LUIS A. SANCHEZ - FERNANDO BELAUNDE TERRY    :    303 - CONG. ARAGON CARREÑO LUIS ANGEL',
 'fechaVisitaTermino': None,
 'edificio': 'LUIS A. SANCHEZ - FERNANDO BELAUNDE TERRY'}

In [34]:
from bs4 import BeautifulSoup
import re

def get_number_from_string(string):
    return int(re.search(r'\d+', string).group())

# A partir de acá, hay que trabajar con BeautifulSoup o algo que parsee HTML
soup = BeautifulSoup(response.text, 'html.parser')
# El primer 'spam' contiene el número de registros
row_count = get_number_from_string(soup.span.text)
row_count

7

In [37]:
rows = soup.find_all('tr')
assert(len(rows) - 1 == row_count)
for row in rows:
    cells = row.find_all('td')
    if len(cells) != 11: # The number of fields...
        print('Esto solo se salta una vez')
        continue
    
    meeting = {
        'visitor_name': cells[2].text,
        'visitor_document': cells[3].text,
        'visitor_entity': cells[4].text,
        'meeting_reason': cells[5].text,
        'public_employee_name': cells[6].text,
        'public_employee_position': cells[7].span.text,
        'office_name': cells[7].span.previousSibling,
        'meeting_start_time': f'{cells[1].text} {cells[8].text}',
        'meeting_end_time': f'{cells[1].text} {cells[9].text}',
        'observation': cells[10].text,
    }
    print(meeting)
    print('')

Esto solo se salta una vez
{'visitor_name': 'ARAPA OBISPO LUIS MIGUEL', 'visitor_document': 'DNI 46967576', 'visitor_entity': 'CONSORCIO ROMERO EIRL', 'meeting_reason': 'OTROS: PROVEEDOR DE CARNE DE AVES', 'public_employee_name': 'YANAPA ZENTENO MARCO', 'public_employee_position': '', 'office_name': 'AREA DE ALIMENTACIóN', 'meeting_start_time': '14/08/2021 08:39', 'meeting_end_time': '14/08/2021 09:26', 'observation': ''}

{'visitor_name': 'VALDERRAMA AGÜERO LILIANA DEL ROCIO', 'visitor_document': 'DNI 10463027', 'visitor_entity': 'INVERSIONES OLIVAL S.A.C.', 'meeting_reason': 'OTROS: INVERSIONES OLIVAL S.A.C.', 'public_employee_name': 'YANAPA ZENTENO MARCO', 'public_employee_position': '', 'office_name': 'AREA DE ALIMENTACIóN', 'meeting_start_time': '14/08/2021 08:29', 'meeting_end_time': '14/08/2021 08:38', 'observation': ''}

{'visitor_name': 'LIZARRAGA MORALES JENRY ROLANDO', 'visitor_document': 'DNI 46231718', 'visitor_entity': 'IDELCOM', 'meeting_reason': 'OTROS: EJECUCION DE SER